# Prática de Aprendizado Supervisionado

**Importando bibliotecas e funções**

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from scipy.io.arff import loadarff
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
import time
import warnings

warnings.filterwarnings("ignore")

**Lendo o arquivo**

In [14]:
def leitura(dataset, nome):

    # Carrega o .arff
    raw_data = loadarff('datasets/extracted/%s/%s.arff' % (dataset, nome))
    # Transforma o .arff em um Pandas Dataframe
    return pd.DataFrame(raw_data[0])
    # Imprime o Dataframe com suas colunas

**Separando em Conjunto de Treino e Teste**

In [3]:
# Com o iloc voce retira as linhas e colunas que quiser do Dataframe, no caso aqui sem as classes
def treinoTeste(df):

    X = df.iloc[:, 0:-1].values

    # Aqui salvamos apenas as classes agora
    y = df['class']
    # Substituimos os valores binários por inteiro
    bow = []
    int_value = 0
    y_aux = []
    for i in y:
      if i in bow:
        y_aux.append(int_value)
      else:
        bow.append(i)
        int_value += 1
        y_aux.append(int_value)
    # Novo y
    y = y_aux
    
    return train_test_split(X,y,test_size=0.2,random_state=327, stratify=y)

**Padronizando os dados com Técnicas de Normalização**

In [4]:
def normalizar(X_train, X_test, selectedNormalization):

    if selectedNormalization == 0:
        return X_train, X_test
    if selectedNormalization == 1:
        scaler = preprocessing.MinMaxScaler()
    if selectedNormalization == 2:
        scaler = preprocessing.StandardScaler()
    if selectedNormalization == 3:
        scaler = preprocessing.MaxAbsScaler()
    if selectedNormalization == 4:
        scaler = preprocessing.RobustScaler()

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    return X_train, X_test

**Treinando os Classificadores**

In [5]:
def treinarClassificadores(classificador, X_train, X_test, y_train, y_test):
        
    cls = classificador
    cls.fit(X_train, y_train)
    t = time.time()
    aux = classificador.predict(X_test)
    f1score   = f1_score(y_test, aux, average = 'macro')
#     precision = precision_score(y_test, aux, average = 'macro')
#     recall    = recall_score(y_test, aux, average = 'macro')
    acc_train = classificador.score(X_train, y_train)
    acc_test  = classificador.score(X_test, y_test)
    tempoExe  = time.time() - t
        
    return acc_train, acc_test, f1score, tempoExe

**Testando o Conjunto de Teste**

In [18]:
def grid(dataset):
    
    normDic = {'0': 'não aplicado', '1': 'MinMaxScaler', '2': 'StandardScaler',
               '3': 'MaxAbsScaler', '4': 'RobustScaler'}
    
    extratores = ['FCTH', 'Gabor', 'GCH', 'LBP', 'LCH', 'Moments', 'PHOG', 'Tamura']
    
    nomes = ['Extrator', 'Normalizador', 'Classificador', 'Tempo', 'Acurácia', 'F1 Score']
    
    classificadores = [(GaussianNB(), 'Gaussian Naive Bayes'), 
                       (LogisticRegression(), 'Logistic Regression'), 
                       (DecisionTreeClassifier(), 'Decision Tree'), 
                       (KNeighborsClassifier(n_neighbors = 3), 'KNN'),
                       (LinearDiscriminantAnalysis(), 'Linear Discriminant Analysis'), 
                       (SVC(), 'SVM'), 
                       (RandomForestClassifier(random_state=42), 'Random Forest'), 
                       (MLPClassifier(alpha=1), 'MLP')]
    
    analise = []
    
    for extrator in extratores:
        
        df = leitura(dataset, extrator)
        
        for norm in range(5):
            X_train, X_test, y_train, y_test = treinoTeste(df)
            X_train, X_test = normalizar(X_train, X_test, norm)
            
            for cls, nome in classificadores:
                dados = treinarClassificadores(cls, X_train, X_test, y_train, y_test)
                acc_train, acc_test, f1score, tempoExe = dados
            
                resultados = [extrator, normDic[str(norm)], nome, tempoExe, acc_test, f1score]
            
                analise.append(resultados)
            
    return pd.DataFrame(analise, columns=nomes).set_index(['Extrator', 'Normalizador', 'Classificador'])

In [31]:
def selecao(resultados):
    
    acc = (resultados['Acurácia'] + resultados['F1 Score']) / 2
    
    return pd.DataFrame(resultados.iloc[np.argmax(acc), :]).T

In [33]:
shapes = grid('Shapes')

In [34]:
melhorShapes = selecao(shapes)

In [35]:
melhorShapes

,,,Tempo,Acurácia,F1 Score
LBP,não aplicado,Random Forest,0.036512,0.983333,0.983323


In [36]:
fruits = grid('Fruits')

In [37]:
melhorFruits = selecao(fruits)

In [38]:
melhorFruits

,,,Tempo,Acurácia,F1 Score
FCTH,não aplicado,KNN,0.368609,0.842105,0.847724


# O porquê
## Fruits

In [39]:
fruits_caracteristicas = leitura('Fruits', 'FCTH')

In [40]:
fruits_caracteristicas = fruits_caracteristicas.describe().T
fruits_caracteristicas.head()

,count,mean,std,min,25%,50%,75%,max
1,1140.0,0.661404,1.014081,0.0,0.0,0.0,1.0,7.0
2,1140.0,1.366667,1.302639,0.0,0.0,1.0,2.0,7.0
3,1140.0,3.264035,1.488476,0.0,2.0,3.0,4.0,7.0
4,1140.0,0.685965,1.131237,0.0,0.0,0.0,1.0,6.0
5,1140.0,1.244737,1.996203,0.0,0.0,0.0,2.0,7.0


In [41]:
np.std(fruits_caracteristicas['max'] - fruits_caracteristicas['min'])

1.848393399610108

FCTH gera características baseado em cor e textura (192 características), as quais apresentam valores desnomalizados. Dataset Alien vs Predator tem diferentes cores e texturas entre as duas classes. Como observado acima, a variação entre as escalas das características são pequenas. Isso pode ter ocasionado a não normalização de nosso dataset

## Shapes

In [43]:
shapes_caracteristicas = leitura('Shapes', 'LBP')

In [44]:
shapes_caracteristicas = shapes_caracteristicas.describe().T
shapes_caracteristicas.head()

,count,mean,std,min,25%,50%,75%,max
1,300.0,10.973333,3.863813,3.0,8.0,11.0,13.0,21.0
2,300.0,1.156667,1.318300,0.0,0.0,1.0,2.0,8.0
3,300.0,4.186667,2.909761,0.0,2.0,3.0,6.0,15.0
4,300.0,0.206667,0.466921,0.0,0.0,0.0,0.0,2.0
5,300.0,1.043333,1.259661,0.0,0.0,1.0,2.0,5.0


In [46]:
np.std(shapes_caracteristicas['max'] - shapes_caracteristicas['min'])

11.778871169610161

O conjunto não apresenta diferença significativa de cores, umas vez que as imagens são pretas e brancas. Assim, o melhor extrator para o dataset foi o LBP o qual só gera características baseado na textura. Outra característica dos dados é que após a aplicação do LBP, as características extraidas não apresentam grande variação quanto à escala, assim a aplicação de um normalizador não foi necessária